In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
import copy


x_col = ['text', 'datetime', 'retweet_count', 'favorite_count', 'place_full_name']
y_col = ['who']

# comma delimited is the default
#df = pd.read_csv(input_file, header = 0)
X_train = pd.read_csv('tweets/tweets-train-data.csv', names = x_col)
y_train = pd.read_csv('tweets/tweets-train-targets.csv', names = y_col)
X_train = X_train.drop('datetime', 1)
X_train = X_train.drop('place_full_name', 1)


X_test = pd.read_csv('tweets/tweets-test-data.csv', names = x_col)
y_test = pd.read_csv('tweets/tweets-test-targets.csv', names = y_col)
X_test = X_test.drop('datetime', 1)
X_test = X_test.drop('place_full_name', 1)

X_test.head()


# Let's check the length of the two sets
#len(X_train), len(X_test)

text  retweet_count  \
0  we dont know why trump and putin praise each o...           7475   
1  is really starting to campaign hard for marcor...           1687   
2  find out where to on caucus night in iowa on 2116           1556   
3  100 true gt is right did call tpp   the gold s...           7419   
4    i took trump university class and saved my home           2879   

  favorite_count  
0          11964  
1           4672  
2           3504  
3          12728  
4           7931

In [2]:
#Change HC -> 0 and DT ->1 into the y dataframes.
dict = {'HC': 0, 'DT': 1}

y_train = y_train.replace({"who": dict})
y_train.head()

y_test = y_test.replace({"who": dict})
y_test.head()


who
0    0
1    1
2    1
3    1
4    1

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.text)
X_train_counts.shape
count_vect.vocabulary_.get(u'algorithm')

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train.who)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [ ]:
#fit text_clf
text_clf.fit(X_train.text, y_train.who)

In [ ]:
#Evaluation of the performance
predicted = text_clf.predict(X_test.text)
np.mean(predicted == y_test.who)

In [5]:
#Lets try with another model. 
#Let’s see if we can do better with a linear support 
#vector machine (SVM), which is widely regarded as one
#of the best text classification algorithms
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


#Create another pipeline
new_text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                              alpha=1e-3, random_state=42,
                              max_iter=5, tol=None)),
])

new_text_clf.fit(X_train.text, y_train.who)
new_predicted = new_text_clf.predict(X_test.text)
np.mean(new_predicted == y_test.who)

/home/filippo/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.8857852265673495

In [19]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

#Convert 0,1 int HC & DT

conf_mat = confusion_matrix(y_test, new_predicted)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=[0,1], yticklabels=[0,1])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()


from sklearn import metrics
report = metrics.classification_report(y_test.values, new_predicted)
print(report)

              precision    recall  f1-score   support

           0       0.86      0.92      0.89       799
           1       0.91      0.85      0.88       812

   micro avg       0.89      0.89      0.89      1611
   macro avg       0.89      0.89      0.89      1611
weighted avg       0.89      0.89      0.89      1611

